# 🎯 MNISTで学ぶAIの「学習」と「賢さ」の測り方

## 📚 この課題で学ぶこと

こんにちは！この課題では、手書き数字の認識を通して、AIがどのようにして賢くなるのか、その「**学習の仕組み**」を情報理論の視点から探ります。

### 🤔 AIはどうやって学習するの？

AIが画像を見て「これは7です！」と答える裏側では、膨大な計算が行われています。AIは最初から賢いわけではありません。たくさんのデータを見て、「正解」と自分の「予測」の「**間違い**」を計算し、その間違いが少しでも小さくなるように自分自身を何度も何度も更新していきます。このプロセスが「**学習**」です。

この「間違いの大きさ」を測る物差しが、今回のテーマである**交差エントロピー**です。

### 📖 今日学ぶキーワード

* **Softmax関数**: AIの出力を、0から9までの各数字である「確率」に変換する魔法の関数 🎲。
* **One-hot エンコーディング**: コンピュータが「正解」を扱いやすい形にするための表現方法。
* **交差エントロピー (損失関数)**: AIの「予測」と「正解」のズレを測る**学習の道しるべ** 📏。AIはこれを小さくすることを目指します。
* **エントロピー**: 学習が終わったAIの予測が、どれくらい「確信」に満ちているか（または「迷っている」か）を測る**信頼度の指標** 💪。
* **不確実性**: AIの「うーん、7かな？1かな？」という"迷い"のこと。エントロピーで数値化できます。

In [ ]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import sys

# seabornスタイルの設定（バージョン対応）
try:
    import seaborn as sns
    plt.style.use('seaborn-v0_8')
except:
    try:
        plt.style.use('seaborn')
    except:
        pass  # デフォルトスタイルを使用

!pip install japanize-matplotlib
import japanize_matplotlib


print("🎉 ライブラリの読み込み完了！")

## 📊 データの準備とビジュアライゼーション

MNISTデータセットを読み込んで、どんな手書き数字があるか見てみましょう。

In [ ]:
# MNISTデータセットの読み込み（サンプル数を制限してCPUでも扱いやすくする）
print("📥 MNISTデータセットを読み込み中...")
mnist = fetch_openml('mnist_784', version=1, parser='auto')

# データサイズを制限（CPUで扱いやすくするため）
n_samples = 10000
X, y = mnist.data[:n_samples], mnist.target[:n_samples]
y = y.astype(int)

print(f"✅ 読み込み完了！ データサイズ: {X.shape}")
print(f"   画像サイズ: 28×28ピクセル")
print(f"   クラス数: {len(np.unique(y))}クラス (0-9の数字)")

japanize_matplotlib.japanize()
# いくつかの手書き数字を表示
fig, axes = plt.subplots(2, 10, figsize=(15, 3))
for i in range(10):
    # 各数字の最初の2つの例を表示
    indices = np.where(y == i)[0][:2]
    for j, idx in enumerate(indices):
        axes[j, i].imshow(X.iloc[idx].values.reshape(28, 28), cmap='gray')
        axes[j, i].set_title(f'Label: {i}', fontsize=10)
        axes[j, i].axis('off')

plt.suptitle('MNISTデータセットの例', fontsize=16)
plt.tight_layout()
plt.show()

## 🧠 シンプルなニューラルネットワークの訓練

手書き数字を分類するモデルを作ります。今回は軽量なMLP分類器を使用します。

In [ ]:
# データの前処理
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 正規化（0-1の範囲に調整）
X_train_scaled = X_train / 255.0
X_test_scaled = X_test / 255.0

print(f"📚 訓練データ: {X_train_scaled.shape[0]}枚")
print(f"🧪 テストデータ: {X_test_scaled.shape[0]}枚")

# ニューラルネットワークの作成と訓練
print("\n🧠 ニューラルネットワークを訓練中...")
model = MLPClassifier(
    hidden_layer_sizes=(128, 64),  # 隠れ層：128個、64個のニューロン
    activation='relu',
    max_iter=10,  # 軽量化のため回数を制限
    random_state=42,
    verbose=True
)

model.fit(X_train_scaled, y_train)

# 訓練完了後の精度確認
train_accuracy = model.score(X_train_scaled, y_train)
test_accuracy = model.score(X_test_scaled, y_test)

print(f"\n✅ 訓練完了！")
print(f"📊 訓練精度: {train_accuracy:.3f}")
print(f"🎯 テスト精度: {test_accuracy:.3f}")

### 📈 訓練過程を覗いてみよう

訓練中のログに `loss` という項目がありましたね。

`Iteration 1, loss = ...`
`Iteration 2, loss = ...`
...
`Iteration 8, loss = ...`

この `loss`（損失）の値が、どんどん小さくなっているのがわかりますか？ これこそが、AIが学習している証拠です。AIは、この **loss = 誤差・損失（今回の場合は誤差関数として交差エントロピーを使用）が最小になるように、内部のパラメータ（重み）を自動で調整している**のです。

**誤差を計算するには出力と正解を比較する**わけですが、出力は確率に変換しなければいけません。どういうことでしょうか。

## 🎲 Softmax関数：AIの出力を「確率」に変換する

ニューラルネットワークは、入力画像に対し、各数字（0〜9）の「っぽさ」を示すスコアを出力します。これでは比較しにくいため、**Softmax関数**を使って、合計が100%（=1.0）になる「**確率**」に変換します。

### 🤖 なぜ確率に変換するの？
後のステップで、AIの予測と正解を比較して「間違いの大きさ（損失）」を計算するために、確率という統一された尺度が必要だからです。

### 📐 Softmaxの計算方法
$$ \text{Softmax}(x_i) = \frac{e^{x_i}}{\sum_{j=1}^{N} e^{x_j}} $$

数式が苦手な方もご安心ください。これは以下の2ステップを行っています。

1.  **スコアを増幅する**: まず `e` という数字（約2.718）を使い、各スコアを増幅させます。これはスコアの差を強調する効果があり、最も「っぽい」スコアを際立たせます。
    > たとえ話：人気投票でAさん10票、Bさん8票を、Aさん60点、Bさん30点のように差を広げて見せるイメージです。
2.  **全体で割って割合を出す**: 増幅した全スコアの合計で、個々の増幅スコアを割ります。これにより、全体におけるそれぞれの「分け前（＝確率）」が計算できます。

**AIが最終的に予測するのは、この確率が最も大きくなったクラスです。**

# 📊 エントロピー：学習済みモデルの「予測の信頼度」を測る

エントロピーは確率分布の「不確かさ」や「予測の難しさ」を表す指標だと学びましたね。確率が全ての選択肢で均等なとき、エントロピーは最大になりました。ここではその知識を応用して、学習が終わったAIの予測がどれだけ「確信」に満ちているか（あるいは「迷っている」か）を見ていきましょう。この「迷いの度合い」を測るのが、ここでのエントロピーの役割です。

学習が終わったモデルが、ある画像を見て「これは9です！」と予測したとします。このとき、モデルはどれくらいその予測に「自信」を持っているのでしょうか？

-   「9である確率99%、他の数字はほぼ0%」 → **自信満々**
-   「9である確率40%、4である確率35%、他は...」 → **かなり迷っている**

この予測の「自信のなさ」や「迷いの度合い」（＝**不確実性**）を数値で表すのが**エントロピー**です。

### 💡 エントロピーのポイント
-   **役割**: 学習済みモデルの**評価・分析**に使う道具。
-   **見方**:
    -   **エントロピーが低い**: 確率がどれか一つに集中している状態。予測への**信頼度が高い**（自信がある）。
    -   **エントロピーが高い**: 確率が複数の選択肢に分散している状態。予測への**信頼度が低い**（迷っている）。

エントロピーは、AIがどんな問題が苦手なのかを分析するのに役立ちますが、**AIが学習する際に直接使う「間違いの物差し」ではありません。** その役割を担うのが、次に出てくる「交差エントロピー」です。

### 🧮 エントロピーの計算式
$$ H(X) = -\sum_{i=1}^{n} p(x_i) \log_2 p(x_i) $$
- $p(x_i)$: ある一つの予測における、クラス $i$ の確率。
- $n$: クラス数。

まずは仮想的なデータで確かめてみます。

In [ ]:
# エントロピー計算関数
def calculate_entropy(probabilities):
    """エントロピーを計算する関数"""
    # 0の確率を除外（log(0)は未定義のため）
    p_filtered = probabilities[probabilities > 0]
    return -np.sum(p_filtered * np.log2(p_filtered))

# 様々な確率分布でエントロピーを計算
distributions = {
    "確実な予測": np.array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    "ほぼ確実": np.array([0.9, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    "やや確実": np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    "不確実": np.array([0.4, 0.3, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    "非常に不確実": np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
}

print("🎯 様々な確率分布のエントロピー")
print("="*50)

japanize_matplotlib.japanize()
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for i, (name, dist) in enumerate(distributions.items()):
    entropy = calculate_entropy(dist)
    print(f"{name:15s}: エントロピー = {entropy:.3f} bit")

    # 各分布を可視化
    if i < len(axes):
        axes[i].bar(range(10), dist, color='lightblue', alpha=0.7)
        axes[i].set_title(f'{name}\nエントロピー: {entropy:.3f} bit')
        axes[i].set_xlabel('クラス')
        axes[i].set_ylabel('確率')
        axes[i].set_ylim(0, 1)
        axes[i].set_xticks(range(10))

# 最後のサブプロットを非表示
axes[-1].axis('off')

plt.tight_layout()
plt.show()

print("\n💡 ポイント:")
print("   - エントロピーが低い = 確実性が高い")
print("   - エントロピーが高い = 不確実性が高い")
print("   - 最大エントロピー = log₂(10) ≈ 3.32 bit（10クラス均等分布）")

# テストデータから予測確率を取得
print("🔍 テストデータで予測確率を計算中...")
pred_probs = model.predict_proba(X_test_scaled)
predictions = model.predict(X_test_scaled)

# 各予測のエントロピーを計算
entropies = []
for prob in pred_probs:
    entropy = calculate_entropy(prob)
    entropies.append(entropy)

entropies = np.array(entropies)

# 予測の正確性を確認
correct_predictions = (predictions == y_test)

print(f"\n📊 予測結果の統計")
print(f"平均エントロピー: {np.mean(entropies):.3f} bit")
print(f"エントロピーの標準偏差: {np.std(entropies):.3f} bit")
print(f"正解率: {np.mean(correct_predictions):.3f}")



## 🎭 予測の信頼度による画像の分類

エントロピーを使って、モデルがどの画像を「確信を持って予測」し、どの画像を「迷って予測」したかを見てみましょう。まずは具体的な例ではなく、仮想的な10クラスの出力をもとに可視化します。

In [ ]:
# Softmax関数の実装
def softmax(x):
    """Softmax関数の実装"""
    exp_x = np.exp(x - np.max(x))  # 数値の安定性のため最大値を引く
    return exp_x / np.sum(exp_x)

# 例：ネットワークのロジット（生のスコア）
raw_scores = np.array([2.0, 1.0, 0.1, 0.5, 0.2, 0.3, 0.8, 0.4, 0.6, 0.9])
probabilities = softmax(raw_scores)

print("🎯 Softmax関数の動作例")
print("生のスコア:", raw_scores)
print("確率:", np.round(probabilities, 2).tolist())
print(f"確率の合計: {np.sum(probabilities):.6f}")

# 可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# 生のスコア
japanize_matplotlib.japanize()
ax1.bar(range(10), raw_scores, color='skyblue', alpha=0.7)
ax1.set_title('生のスコア（logits）')
ax1.set_xlabel('クラス (0-9)')
ax1.set_ylabel('スコア')
ax1.set_xticks(range(10))

# 確率
ax2.bar(range(10), probabilities, color='lightcoral', alpha=0.7)
ax2.set_title('Softmax 後の確率')
ax2.set_xlabel('クラス (0-9)')
ax2.set_ylabel('確率')
ax2.set_xticks(range(10))
ax2.set_ylim(0, 1)

plt.tight_layout()
plt.show()

print(f"\n🎯 予測クラス: {np.argmax(probabilities)} (確率: {np.max(probabilities):.3f})")

### 🤔 課題1: ロジットと softmax

- クラス 3 に分類されるよう、上記のコードにおけるロジットを修正してください。


- 正解がクラス 3 だったとしたら、One-hot エンコーディングはどうなりますか？

回答：

## 🎯 実際の予測でのエントロピー分析

訓練したモデルを使って、実際の予測におけるエントロピーを調べてみましょう。

In [ ]:
# エントロピーでソートして、確信度の高い・低い画像を抽出
sorted_indices = np.argsort(entropies)

# 最も確信度の高い画像（エントロピーが低い）
most_confident_indices = sorted_indices[:10]
# 最も確信度の低い画像（エントロピーが高い）
least_confident_indices = sorted_indices[-10:]

def show_predictions(indices, title, entropies, pred_probs, X_test, y_test, predictions):
    """予測結果を可視化する関数"""

    japanize_matplotlib.japanize()
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    axes = axes.flatten()

    for i, idx in enumerate(indices):
        # 画像表示
        axes[i].imshow(X_test.iloc[idx].values.reshape(28, 28), cmap='gray')

        # タイトル情報
        true_label = y_test.iloc[idx]
        pred_label = predictions[idx]
        entropy = entropies[idx]
        max_prob = np.max(pred_probs[idx])

        # 正解・不正解の判定
        is_correct = "[〇]" if true_label == pred_label else "[×]"

        axes[i].set_title(f'{is_correct} 正解:{true_label} 予測:{pred_label}\n'
                         f'エントロピー:{entropy:.2f}\n確率:{max_prob:.2f}',
                         fontsize=10)
        axes[i].axis('off')

    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

# 確信度の高い予測
print("最も確信度の高い予測（エントロピーが小さい）")
print("="*50)
show_predictions(most_confident_indices, "最も確信度の高い予測",
                entropies, pred_probs, X_test_scaled, y_test, predictions)

# 確信度の低い予測
print("\n最も確信度の低い予測（エントロピーが大きい）")
print("="*50)
show_predictions(least_confident_indices, "最も確信度の低い予測",
                entropies, pred_probs, X_test_scaled, y_test, predictions)

# 統計情報 - インデックスの問題を修正
# y_testのインデックスをリセットして連続にする
y_test_reset = y_test.reset_index(drop=True)
correct_predictions_reset = (predictions == y_test_reset)

high_conf_correct = np.mean(correct_predictions_reset[most_confident_indices])
low_conf_correct = np.mean(correct_predictions_reset[least_confident_indices])

print(f"\n📊 統計情報")
print(f"高確信度予測の正解率: {high_conf_correct:.2f}")
print(f"低確信度予測の正解率: {low_conf_correct:.2f}")

In [ ]:
# エントロピーの分布を可視化
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# 全体のエントロピー分布
axes[0, 0].hist(entropies, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].axvline(np.mean(entropies), color='red', linestyle='--', label=f'平均: {np.mean(entropies):.3f}')
axes[0, 0].set_title('全予測のエントロピー分布')
axes[0, 0].set_xlabel('エントロピー (bit)')
axes[0, 0].set_ylabel('頻度')
axes[0, 0].legend()

# 正解・不正解別のエントロピー分布
correct_entropies = entropies[correct_predictions]
incorrect_entropies = entropies[~correct_predictions]

axes[0, 1].hist(correct_entropies, bins=30, alpha=0.7, color='green', label='正解', density=True)
axes[0, 1].hist(incorrect_entropies, bins=30, alpha=0.7, color='red', label='不正解', density=True)
axes[0, 1].set_title('正解・不正解別エントロピー')
axes[0, 1].set_xlabel('エントロピー (bit)')
axes[0, 1].set_ylabel('密度')
axes[0, 1].legend()

print(f"\n🎯 正解時の平均エントロピー: {np.mean(correct_entropies):.3f} bit")
print(f"❌ 不正解時の平均エントロピー: {np.mean(incorrect_entropies):.3f} bit")

# 最大確率とエントロピーの関係
max_probs = np.max(pred_probs, axis=1)
axes[1, 0].scatter(entropies, max_probs, alpha=0.5, s=1)
axes[1, 0].set_title('エントロピーと最大確率の関係')
axes[1, 0].set_xlabel('エントロピー (bit)')
axes[1, 0].set_ylabel('最大確率')

# クラス別エントロピー
class_entropies = []
for i in range(10):
    class_mask = (y_test == i)
    if np.sum(class_mask) > 0:
        class_entropies.append(np.mean(entropies[class_mask]))
    else:
        class_entropies.append(0)

axes[1, 1].bar(range(10), class_entropies, color='lightcoral', alpha=0.7)
axes[1, 1].set_title('クラス別 平均エントロピー')
axes[1, 1].set_xlabel('数字クラス')
axes[1, 1].set_ylabel('平均エントロピー (bit)')
axes[1, 1].set_xticks(range(10))

plt.tight_layout()
plt.show()

print("\n💡 観察ポイント:")
print("   - 正解時のエントロピーは低い傾向")
print("   - 不正解時のエントロピーは高い傾向")
print("   - エントロピーが低い = モデルが確信を持った予測")
print("   - エントロピーが高い = モデルが迷った予測")

### 🤔 課題 2: エントロピーと正解・不正解

エントロピーが大きいほど正解しやすいですか、それとも間違えやすいですか？

上記のグラフを読み取りつつ、理由とともに回答してください。

回答：

# 🎯 交差エントロピー：学習の道しるべとなる「損失関数」

いよいよ、AIが学習するための最重要パーツである**損失関数**です。AIは、**予測と正解の「ズレ」**を計算し、それを最小化するのでしたね。この「ズレ」を測る物差しが**交差エントロピー**です。


### 🧮 交差エントロピーの計算方法
$$ H(p, q) = -\sum_{i=1}^{n} p(x_i) \log q(x_i) $$
- $p(x_i)$: **正解**の確率分布（One-hotエンコーディング）
- $q(x_i)$: **予測**の確率分布（Softmaxの出力）

この計算が賢いのは、**実質的に「正解」だったクラスに対して、AIがどれくらいの確率を予測したかだけを見ている**点です。

例えば正解が「7」の場合、この計算は最終的に `–log(AIが7と予測した確率)` というシンプルな形になります。

-   AIが「7である確率95%」と高く予測すれば、`–log(0.95)` は**小さな値**になり、AIは褒められます（損失が小さい）。
-   AIが「7である確率5%」と低く予測すれば、`–log(0.05)` は**大きな値**になり、ペナルティを受けます（損失が大きい）。

> たとえ話：🎯 ダーツの的当てゲームです。交差エントロピーは、的のど真ん中（正解）から矢が外れた**距離（＝間違いの大きさ）**です。AIはこの距離が常に最小になるよう、投げるフォームを何度も微調整（学習）していくのです。

### ⚙️ 誤差逆伝播法：AIが「反省」して賢くなる仕組み

さて、AIは「間違いの大きさ（＝交差エントロピー損失）」を計算できるようになりました。
では、その数値をどうやってAIの成長、つまり「学習」に繋げるのでしょうか？

その答えが、ニューラルネットワークの学習における心臓部、**誤差逆伝播法（Backpropagation）**です。

これは、**最終的な「誤差」について、出力側から入力側へさかのぼり、ネットワーク内部の全ての「重み」を少しずつ賢い方向に修正していく**手法です。

#### 🤔 どんな仕組み？

学習の1サイクルは、大きく4つのステップで構成されます。

1.  ➡️ **予測（順伝播 - Forward Propagation）**
    まず、入力データ（画像）がネットワークの入り口から入ります。データは層を一つずつ進むたびに、各ニューロンの**重み**によって計算され、変換されていきます。そして最終的に「**出力**（予測された確率）」となって出てきます。この一方向の流れを**順伝播**と呼びます。

2.  😠 **間違いの計算（損失の計算）**
    順伝播によって得られた「出力（予測）」と「正解ラベル」を比べ、その**間違いの大きさ**を**交差エントロピー**のような「誤差関数」で計算します。その結果が「誤差」です。

3.  👈 **間違いの原因をさかのぼる（誤差逆伝播 - Backpropagation）**
    計算した「間違い」は、元をたどればネットワーク内部の無数の**重み**が少しずつ不適切な設定だったせいです。誤差逆伝播法は、まず**出力に最も近い層の重み**が、最終的な間違いにどれだけ影響したかを計算します。
    次に、その結果を利用して、**一つ手前の層の重み**がどれだけ影響したかを計算します。このプロセスを、**出力から入力へ**と逆方向に繰り返していきます。これにより、**ネットワーク内の全ての重みを「**間違いが少しでも小さくなる方向**」に、ほんの僅かだけ修正（更新）します。

この**「①予測 → ②間違いを計算 → ③原因をさかのぼる → ④重みを修正」**というサイクルを何千、何万回と繰り返すことで、ネットワーク全体の重みが最適化され、AIは賢くなっていくのです。

最初の訓練ログで `loss` が少しずつ減っていたのは、まさにこの誤差逆伝播法によってAIが「反省と改善」を繰り返していたからです。

In [ ]:
# 交差エントロピー計算関数
def cross_entropy(y_true, y_pred):
    """交差エントロピーを計算する関数"""
    # 数値安定性のため、非常に小さい値で下限を設定
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.sum(y_true * np.log(y_pred))

# One-hot エンコーディングの例
print("🎯 One-hot エンコーディングの例")
print("="*40)

for digit in [0, 3, 7, 9]:
    one_hot = np.zeros(10)
    one_hot[digit] = 1
    print(f"数字 {digit}: {one_hot}")

# 交差エントロピーの実例
print("\n🧮 交差エントロピーの計算例")
print("="*40)

# 正解: 数字「3」
true_label = np.zeros(10)
true_label[3] = 1.0

# 様々な予測パターン
prediction_cases = {
    "完璧な予測": np.array([0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    "良い予測": np.array([0.05, 0.05, 0.05, 0.7, 0.05, 0.05, 0.0, 0.0, 0.0, 0.0]),
    "まずまずの予測": np.array([0.1, 0.1, 0.1, 0.4, 0.1, 0.1, 0.05, 0.05, 0.0, 0.0]),
    "悪い予測": np.array([0.2, 0.2, 0.2, 0.2, 0.1, 0.05, 0.03, 0.02, 0.0, 0.0]),
    "非常に悪い予測": np.array([0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
}

print(f"正解ラベル: {true_label}\n")

results = []
for name, pred in prediction_cases.items():
    ce = cross_entropy(true_label, pred)
    entropy = calculate_entropy(pred)
    confidence = np.max(pred)
    results.append((name, ce, entropy, confidence))
    print(f"{name:15s}: 交差エントロピー = {ce:.3f}, エントロピー = {entropy:.3f}, 最大確率 = {confidence:.3f}")

# 可視化
japanize_matplotlib.japanize()
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for i, (name, pred) in enumerate(prediction_cases.items()):
    if i < len(axes):
        ce = cross_entropy(true_label, pred)
        axes[i].bar(range(10), pred, color='lightgreen', alpha=0.7)
        axes[i].bar(3, pred[3], color='red', alpha=0.8)  # 正解クラスを強調
        axes[i].set_title(f'{name}\n交差エントロピー: {ce:.3f}')
        axes[i].set_xlabel('クラス')
        axes[i].set_ylabel('予測確率')
        axes[i].set_ylim(0, 1)
        axes[i].set_xticks(range(10))

# 最後のサブプロットを非表示
axes[-1].axis('off')

plt.tight_layout()
plt.show()

print("\n💡 ポイント:")
print("   - 正解クラスの確率が高いほど交差エントロピーは小さい")
print("   - 正解クラスの確率が低いほど交差エントロピーは大きい")
print("   - 交差エントロピーが小さい = より良い予測")

# テストデータのone-hotエンコーディング
def to_one_hot(labels, num_classes=10):
    """ラベルをone-hotエンコーディングに変換"""
    one_hot = np.zeros((len(labels), num_classes))
    one_hot[np.arange(len(labels)), labels] = 1
    return one_hot

# テストデータのone-hot化
y_test_one_hot = to_one_hot(y_test)

# 各サンプルの交差エントロピーを計算
cross_entropies = []
for i in range(len(y_test)):
    ce = cross_entropy(y_test_one_hot[i], pred_probs[i])
    cross_entropies.append(ce)

cross_entropies = np.array(cross_entropies)

print(f"📊 交差エントロピーの統計")
print(f"平均交差エントロピー: {np.mean(cross_entropies):.3f}")
print(f"交差エントロピーの標準偏差: {np.std(cross_entropies):.3f}")
print(f"最小交差エントロピー: {np.min(cross_entropies):.3f}")
print(f"最大交差エントロピー: {np.max(cross_entropies):.3f}")

# 正解・不正解別の交差エントロピー
correct_ce = cross_entropies[correct_predictions]
incorrect_ce = cross_entropies[~correct_predictions]

print(f"\n✅ 正解時の平均交差エントロピー: {np.mean(correct_ce):.3f}")
print(f"❌ 不正解時の平均交差エントロピー: {np.mean(incorrect_ce):.3f}")

### 🤔 課題 3: One-hot エンコーディングと交差エントロピー

上記のそれぞれのグラフでは、**何と比較して** 交差エントロピーを求めていますか？比較対象となるベクトルを書いてください。

回答： $ [ 0, この続きを埋める ] $

## 🔍 実際のテストデータでの交差エントロピー分析

訓練したモデルの実際の予測について、交差エントロピーを詳しく分析してみましょう。

In [ ]:
# 可視化
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# 交差エントロピーの分布
axes[0, 0].hist(cross_entropies, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].axvline(np.mean(cross_entropies), color='red', linestyle='--', label=f'平均: {np.mean(cross_entropies):.3f}')
axes[0, 0].set_title('交差エントロピーの分布')
axes[0, 0].set_xlabel('交差エントロピー')
axes[0, 0].set_ylabel('頻度')
axes[0, 0].set_yscale('log')
axes[0, 0].legend()

# 正解・不正解別の交差エントロピー
axes[0, 1].hist(correct_ce, bins=5, alpha=0.7, color='green', label='正解', density=True)
axes[0, 1].hist(incorrect_ce, bins=30, alpha=0.7, color='red', label='不正解', density=True)
axes[0, 1].set_title('正解・不正解別交差エントロピー')
axes[0, 1].set_xlabel('交差エントロピー')
axes[0, 1].set_ylabel('密度')
axes[0, 1].set_yscale('log')
axes[0, 1].legend()

# 交差エントロピーとエントロピーの関係
axes[1, 0].scatter(entropies, cross_entropies, alpha=0.5, s=1)
axes[1, 0].set_title('エントロピー vs 交差エントロピー')
axes[1, 0].set_xlabel('エントロピー')
axes[1, 0].set_ylabel('交差エントロピー')

# 最大確率と交差エントロピーの関係
axes[1, 1].scatter(max_probs, cross_entropies, alpha=0.5, s=1)
axes[1, 1].set_title('最大確率 vs 交差エントロピー')
axes[1, 1].set_xlabel('最大確率')
axes[1, 1].set_ylabel('交差エントロピー')

plt.tight_layout()
plt.show()

### 🤔  課題 5: 不確実性による画像の難易度分析

テストデータの中から、特定の数字（あなたが選んだ数字）について、エントロピーが高い（予測が困難な）画像と低い（予測が容易な）画像を比較分析してください。

- 予測しやすい画像の特徴は何ですか？

答え:

- なぜモデルは一部の画像で迷ったと思いますか？

答え:

#### 💡 観察ポイント:"
- 確信度の高い予測は正解率が高い傾向
- 確信度の低い予測では字が汚い、紛らわしいケースが多い
- エントロピーは予測の信頼性の指標として使える

In [ ]:
# 📝 課題3: 数字の難易度分析

# TODO: あなたが分析したい数字を選んでください（0-9）
target_digit = 8  # ここを変更してください

print(f"📝 課題3: 数字'{target_digit}'の難易度分析")
print("="*50)

# y_testをNumPy配列に変換（まだされていない場合）
y_test_array = np.array(y_test)
correct_predictions_array = (predictions == y_test_array)

# 選択した数字のインデックスを取得
target_indices = np.where(y_test_array == target_digit)[0]
print(f"数字{target_digit}のテスト画像数: {len(target_indices)}枚")

if len(target_indices) > 0:
    # その数字のエントロピーと予測結果を取得
    target_entropies = entropies[target_indices]
    target_predictions = predictions[target_indices]
    target_correct = correct_predictions_array[target_indices]
    target_probs = pred_probs[target_indices]

    # 統計情報
    print(f"\n📊 数字{target_digit}の統計:")
    print(f"   平均エントロピー: {np.mean(target_entropies):.3f}")
    print(f"   正解率: {np.mean(target_correct):.3f}")
    print(f"   最小エントロピー: {np.min(target_entropies):.3f}")
    print(f"   最大エントロピー: {np.max(target_entropies):.3f}")

    # エントロピーでソート
    sorted_target_indices = np.argsort(target_entropies)

    # 最も予測しやすい画像（エントロピー低）
    easy_indices = target_indices[sorted_target_indices[:5]]
    # 最も予測しにくい画像（エントロピー高）
    hard_indices = target_indices[sorted_target_indices[-5:]]

    # 可視化関数
    def analyze_difficulty(indices, title):
        fig, axes = plt.subplots(1, 5, figsize=(15, 3))

        for i, idx in enumerate(indices):
            axes[i].imshow(X_test_scaled.iloc[idx].values.reshape(28, 28), cmap='gray')

            entropy = entropies[idx]
            pred_label = predictions[idx]
            max_prob = np.max(pred_probs[idx])
            is_correct = "[〇]" if y_test_array[idx] == pred_label else "[×]"

            axes[i].set_title(f'{is_correct} 予測:{pred_label}\nエントロピー:{entropy:.2f}\n確率:{max_prob:.2f}',
                             fontsize=10)
            axes[i].axis('off')

        plt.suptitle(title, fontsize=14)
        plt.tight_layout()
        plt.show()

    # 結果表示
    print(f"\n🟢 予測しやすい数字{target_digit}の画像（エントロピー低）:")
    analyze_difficulty(easy_indices, f"予測しやすい数字 {target_digit}")

    print(f"\n🔴 予測しにくい数字{target_digit}の画像（エントロピー高）:")
    analyze_difficulty(hard_indices, f"予測しにくい数字 {target_digit}")

else:
    print(f"⚠️ 数字{target_digit}のテストデータが見つかりませんでした。")

### 🤔 課題 6: 混同行列と不確実性の関係

混同行列（Confusion Matrix）を作成し、どの数字同士が間違えやすいかを読み取ってください。どのような誤りパターンが多いですか？

In [ ]:
# 📝 課題4: 混同行列と不確実性の関係

print("📝 課題4: 混同行列と不確実性の関係")
print("="*50)

# 混同行列の作成と可視化
cm = confusion_matrix(y_test, predictions)

import matplotlib.pyplot as plt
import numpy as np


try:
    import seaborn as sns

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=range(10), yticklabels=range(10))
    plt.title('混同行列（Confusion Matrix）')
    plt.xlabel('予測ラベル')
    plt.ylabel('正解ラベル')
    plt.show()

except ImportError:
    print("Seaborn が見つかりません。matplotlib を使用して代替表示します。")

    plt.figure(figsize=(10, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('混同行列（Confusion Matrix）')
    plt.colorbar()

    tick_marks = np.arange(10)
    plt.xticks(tick_marks, tick_marks)
    plt.yticks(tick_marks, tick_marks)

    thresh = cm.max() / 2
    for i in range(10):
        for j in range(10):
            plt.text(j, i, format(cm[i, j], 'd'),
                     ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.xlabel('予測ラベル')
    plt.ylabel('正解ラベル')
    plt.tight_layout()
    plt.show()


# 最も間違えやすい組み合わせを探す
print("\n🔍 最も間違えやすい数字の組み合わせ:")
errors = []
for i in range(10):
    for j in range(10):
        if i != j and cm[i, j] > 0:
            errors.append((cm[i, j], i, j))

errors.sort(reverse=True)
for count, true_label, pred_label in errors[:10]:
    print(f"   {true_label} → {pred_label}: {count}回の間違い")

# 高エントロピー予測での間違いパターン分析
high_entropy_threshold = np.percentile(entropies, 80)  # 上位20%の高エントロピー
high_entropy_mask = entropies > high_entropy_threshold
high_entropy_incorrect = high_entropy_mask & (~correct_predictions)

print(f"\n📈 高エントロピー予測（>{high_entropy_threshold:.2f}）での間違い分析:")
if np.sum(high_entropy_incorrect) > 0:
    high_entropy_true = y_test[high_entropy_incorrect]
    high_entropy_pred = predictions[high_entropy_incorrect]

    # 高エントロピーでの間違いパターン
    from collections import Counter
    error_patterns = Counter(zip(high_entropy_true, high_entropy_pred))

    print("   高エントロピー時の主な間違いパターン:")
    for (true_val, pred_val), count in error_patterns.most_common(5):
        print(f"     {true_val} → {pred_val}: {count}回")

# 🎓 総括課題: 情報理論と不確実性のまとめ

今回の課題を通して学んだことを、以下のキーワードを**すべて含めて**、それぞれの**関係性**がわかるように自分の言葉でまとめてください。

### 🔑 必須キーワード
- **Softmax関数**
- **One-hot エンコーディング**
- **エントロピー**
- **交差エントロピー**
- **損失関数**

### 💡 まとめの観点
1.  AIの「学習」というプロセスにおいて、それぞれのキーワードが**どの段階でどのような役割**を果たしているか。
2.  **エントロピー**と**交差エントロピー**の**重要な違い**は何か。

## 🎉 おつかれさまでした！
